In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install kaggle
!gdown --id 14lE9PtdFcSuZIW84MZSDpjTp46E9wnDV
!mkdir ~/.kaggle; cp /content/kaggle.json ~/.kaggle/kaggle.json
!kaggle datasets download -d tapakah68/audio-dataset
!unzip audio-dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: audio_dataset/audio_files/f5459927-b691-41f8-a80d-2b4eb85930e8.wav  
  inflating: audio_dataset/audio_files/f545af60-3f1d-46fa-aea4-fe526bf0095c.wav  
  inflating: audio_dataset/audio_files/f5469a4e-3f82-4db6-bce1-91c3e608a54b.wav  
  inflating: audio_dataset/audio_files/f547012e-b4e8-4a7e-96c7-8222fcb0fad3.wav  
  inflating: audio_dataset/audio_files/f5477f37-e23f-4c0c-9cd5-e5cf3835e2a0.wav  
  inflating: audio_dataset/audio_files/f5487773-4c49-4c50-8db8-5a043281566a.wav  
  inflating: audio_dataset/audio_files/f54a0ef1-1a59-468d-b731-461453ed6e98.wav  
  inflating: audio_dataset/audio_files/f54a57f6-598c-46ad-bc29-199b2983539f.wav  
  inflating: audio_dataset/audio_files/f54b13c3-063a-4b24-8d62-f426fb4ec7f2.wav  
  inflating: audio_dataset/audio_files/f54b7f6c-4029-491a-91d3-75e1c3f0efef.wav  
  inflating: audio_dataset/audio_files/f54b9827-f8f5-4750-8e05-a69bdf8fc327.wav  
  inflating: audio_dataset/aud

In [ ]:
!pip install iteration_utilities

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 283 kB 21.3 MB/s 


In [ ]:
import pandas as pd
import numpy as np

df_info = pd.read_csv('audio_dataset/df.csv')
df_info.head()

,text,audio_id,worker_id,status,version,pool_id,task_id,path,duration,frames,rate
0,"По его словам, на вчерашний вечер у Донбанка «...",2e5fc762-6b7c-4168-8dfd-44e90e908de3,bd83ddb2596e455951c3838ed9061b84,APPROVED,1,12233816,0000baac58--5e813e8b44e47001255de14a,audio_files/2e5fc762-6b7c-4168-8dfd-44e90e908d...,13.02,208320.0,16000.0
1,"Этот процесс так захватывает человека, что он ...",93b47a6a-504c-41e8-b420-1ed5b8360d0d,bd83ddb2596e455951c3838ed9061b84,APPROVED,1,12233816,0000baac58--5e813e8b44e47001255de14a,audio_files/93b47a6a-504c-41e8-b420-1ed5b8360d...,15.02,240320.0,16000.0
2,Это крупнейшее левое движение Западного полуша...,19588ca3-9249-43cd-bc55-374432dffdbd,bd83ddb2596e455951c3838ed9061b84,APPROVED,1,12233816,0000baac58--5e813e8b44e47001255de14a,audio_files/19588ca3-9249-43cd-bc55-374432dffd...,15.48,247680.0,16000.0
3,А результаты проекта будут представлены на еже...,b8155462-6b72-46b6-a28c-71b0ee8fcd84,bd83ddb2596e455951c3838ed9061b84,APPROVED,1,12233816,0000baac58--5e813e8b44e47001255de14a,audio_files/b8155462-6b72-46b6-a28c-71b0ee8fcd...,14.32,229120.0,16000.0
4,За последний месяц лидеры двух стран встречают...,82b5db50-ec4c-493e-b071-e15ab179edda,bd83ddb2596e455951c3838ed9061b84,APPROVED,1,12233816,0000baac58--5e813e8b44e47001255de14a,audio_files/82b5db50-ec4c-493e-b071-e15ab179ed...,11.80,188800.0,16000.0


In [ ]:
df_info.groupby('worker_id').size().min(), df_info.groupby('worker_id').size().max()

(320, 4000)

In [ ]:
df_info['worker_id'].nunique()

66

In [ ]:
%%writefile utils.py
import numpy as np
import soundfile as sf
from scipy.signal import resample_poly


def read_scaled_wav(path, scaling_factor, downsample_8K=False):
    samples, sr_orig = sf.read(path)

    if len(samples.shape) > 1:
        samples = samples[:, 0]

    if downsample_8K:
        samples = resample_poly(samples, 8000, sr_orig)
    samples *= scaling_factor
    return samples


def wavwrite_quantize(samples):
    return np.int16(np.round((2 ** 15) * samples))


def quantize(samples):
    int_samples = wavwrite_quantize(samples)
    return np.float64(int_samples) / (2 ** 15)


def wavwrite(file, samples, sr):
    """This is how the old Matlab function wavwrite() quantized to 16 bit.
    We match it here to maintain parity with the original dataset"""
    int_samples = wavwrite_quantize(samples)
    sf.write(file, int_samples, sr, subtype='PCM_16')


def append_or_truncate(s1_samples, s2_samples, noise_samples, min_or_max='max', start_samp_16k=0, downsample=False):
    if downsample:
        speech_start_sample = start_samp_16k // 2
    else:
        speech_start_sample = start_samp_16k

    speech_end_sample = speech_start_sample + len(s1_samples)

    if min_or_max == 'min':
        noise_samples = noise_samples[speech_start_sample:speech_end_sample]
    else:
        s1_append = np.zeros_like(noise_samples)
        s2_append = np.zeros_like(noise_samples)
        s1_append[speech_start_sample:speech_end_sample] = s1_samples
        s2_append[speech_start_sample:speech_end_sample] = s2_samples
        s1_samples = s1_append
        s2_samples = s2_append

    return s1_samples, s2_samples, noise_samples


def fix_length(s1, s2, min_or_max='max'):
    # Fix length
    if min_or_max == 'min':
        utt_len = np.minimum(len(s1), len(s2))
        s1 = s1[:utt_len]
        s2 = s2[:utt_len]
    else:  # max
        utt_len = np.maximum(len(s1), len(s2))
        s1 = np.append(s1, np.zeros(utt_len - len(s1)))
        s2 = np.append(s2, np.zeros(utt_len - len(s2)))
    return s1, s2

Writing utils.py


In [ ]:
%%writefile create_wav_2speakers.py
import os
import numpy as np
import pandas as pd
import argparse
from utils import wavwrite, read_scaled_wav, fix_length


S1_DIR = 's1'
S2_DIR = 's2'
MIX_DIR = 'mix'

FILELIST_STUB = os.path.join('train', 'mix_2_spk_filenames_{}.csv')


def create_mix(data_root, s1_path, s2_path, save_path, datalen_dir='min', sr_str='16k'):
    if sr_str == '8k':
        sr = 8000
        downsample = True
    else:
        sr = 16000
        downsample = False

    s1 = read_scaled_wav(os.path.join(data_root, s1_path + '.wav'), 1, downsample)
    s2 = read_scaled_wav(os.path.join(data_root, s2_path + '.wav'), 0.8, downsample)

    s1, s2 = fix_length(s1, s2, datalen_dir)
    mix = s1 + s2
    wavwrite(os.path.join('mixes/', save_path + '.wav'), mix, sr)

Writing create_wav_2speakers.py


In [ ]:
from iteration_utilities import random_combination
from tqdm.notebook import tqdm

from utils import read_scaled_wav, wavwrite
from create_wav_2speakers import create_mix
import os
import numpy as np

def create_mixes_k(df_info, id_num=10, samples_num=100, k=2):
    bad_names = ['3296081fb8ec6b687b49d62e686fa3ea', '05a177d28571adf2bf84bc67961db589', '08e1f623bdcd244cee0b428cd338e4f5', '71a8454e732be2ccb64ce3c661a6ad1d']
    worker_list = df_info['worker_id'].unique()
    worker_list_chosen = worker_list[np.random.permutation(np.arange(len(worker_list)))[:id_num]]
    for i, worker_id in tqdm(enumerate(worker_list_chosen), total=id_num):
        if worker_id in bad_names:
            pass
        # if not os.path.exists(f'mixes/{worker_id}'):
        #     os.makedirs(f'mixes/{worker_id}')
        if not os.path.exists(f'mixes/'):
            os.makedirs(f'mixes/')

        origin_path = df_info[df_info['worker_id'] == worker_id]['audio_id'].values[0]
        origin = read_scaled_wav('audio_dataset/audio_files/' + origin_path + '.wav', 1, False)
        
        background_all = df_info[df_info['worker_id'] != worker_id]['audio_id'].values
        background_all_names = df_info[df_info['worker_id'] != worker_id]
        background_all_names = background_all_names[['audio_id', 'worker_id']]

        # background_all
        
        for j, target in enumerate(random_combination(df_info[df_info['worker_id'] == worker_id]['audio_id'].values, samples_num)):
            if target in bad_names:
              pass
            target_wav = read_scaled_wav('audio_dataset/audio_files/' + target + '.wav', 1, False)
            
            
            # background_tracks = random_combination(background_all, k-1)
            rand_indeces = random_combination(np.arange(len(background_all_names)), k-1)
            background_tracks_names = background_all_names.iloc[rand_indeces]
            # print(background_tracks_names)
            # background_tracks = background_tracks_names['audio_id'].values
            # for background in background_tracks:
            for index in rand_indeces:
                # print(background)
                if background_all_names.iloc[index]['worker_id'] in bad_names:
                    # print('You shall not PASS!!!!!!!')
                    pass
                noise_name = background_all_names.iloc[index]['worker_id']
                wavwrite(f'mixes/{worker_id}_{noise_name}-target.wav', target_wav, 16_000)
                wavwrite(f'mixes/{worker_id}_{noise_name}-ref_emb.wav', origin, 16_000)
                create_mix('audio_dataset/audio_files', target, background_all_names.iloc[index]['audio_id'], f'{worker_id}_{noise_name}-mixed' ,'min');
                # wavwrite(f'mixes/{worker_id}/{worker_id}_{noise_name}-target.wav', target_wav, 16_000)
                # wavwrite(f'mixes/{worker_id}/{worker_id}_{noise_name}-ref_emb.wav', origin, 16_000)
                # create_mix('audio_dataset/audio_files', target, background_all_names.iloc[index]['audio_id'], f'{worker_id}/{worker_id}_{noise_name}_-mixed' ,'min');

In [ ]:
!rm -rf mixes

In [ ]:
create_mixes_k(df_info, id_num=66, samples_num=7, k=2)

  0%|          | 0/66 [00:00<?, ?it/s]

In [ ]:
!zip -r mixes.zip  mixes/

Выходные данные были обрезаны до нескольких последних строк (5000).
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/noisy_47.wav (deflated 16%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/noisy_97.wav (deflated 8%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/noisy_46.wav (deflated 27%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/clean_80.wav (deflated 25%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/clean_6.wav (deflated 34%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/clean_47.wav (deflated 29%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/noisy_61.wav (deflated 9%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/clean_81.wav (deflated 29%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/noisy_36.wav (deflated 23%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/clean_13.wav (deflated 27%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/noisy_57.wav (deflated 17%)
  adding: mixes/d5f7117d2f07eba952f99ce38d29031b/clean_98.wav (deflated 32%)
  adding: m

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/Sound")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.append(str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Mounted at /content/drive


In [ ]:
cp -r /content/mixes/ /content/drive/MyDrive/Sound/mixes_small_better/

In [ ]:
! ls -l

total 20593136
drwxr-xr-x 3 root root        4096 Jun 30 10:59 audio_dataset
-rw-r--r-- 1 root root 18978744658 Jun 30 10:54 audio-dataset.zip
-rw-r--r-- 1 root root         748 Jun 30 10:59 create_wav_2speakers.py
drwx------ 5 root root        4096 Jun 30 11:34 drive
-rw-r--r-- 1 root root          71 Jun 30 10:50 kaggle.json
-rw-r--r-- 1 root root  2108585425 Jun 30 11:05 mixes.zip
drwxr-xr-x 2 root root        4096 Jun 30 10:59 __pycache__
drwxr-xr-x 1 root root        4096 Jun 15 13:42 sample_data
-rw-r--r-- 1 root root        2009 Jun 30 10:59 utils.py
